# Exercice Pandas BigML Kaggle

imports

In [1]:
import pandas as pd
import bigml.api
import kaggle

set bigML API Key

In [2]:
api = bigml.api.BigML(project='project/5db1644859f5c33b3c00076c')

import from scv file

In [3]:
source = api.create_source('./cs-training.csv')
origin_dataset = api.create_dataset(source)

test_source = api.create_source('./cs-test.csv')
kaggle_test_dataset = api.create_dataset(test_source)

SystemExit: ERROR: cannot read training set

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
train_dataset = api.create_dataset(
    origin_dataset, {"name": "Dataset Name | Training",
                     "sample_rate": 0.8, "seed": "my seed"})
test_dataset = api.create_dat
aset(
    origin_dataset, {"name": "Dataset Name | Test",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

Création du modèle

In [ ]:
ensemble = api.create_ensemble(train_dataset, {"objective_field":"SeriousDlqin2yrs"})

In [ ]:
evaluation = api.create_evaluation(ensemble, test_dataset)

In [ ]:
batch_prediction = api.create_batch_prediction(ensemble, kaggle_test_dataset, {
    "name": "my batch prediction", "all_fields": True,
    "header": True,
    "confidence": True,
    "probabilities": True})

In [4]:
api.download_batch_prediction('batchprediction/5db1659f7811dd7f2d008752',
                             filename='./prediction.csv')


'./prediction.csv'

## Creation du Dataframe Pandas

In [5]:
df = pd.read_csv('./prediction.csv', header=0, index_col=0)
df

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs.1,confidence,0 probability,1 probability
field1,,,,,,,,,,,,,,,
1,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0,0,0.92435,0.94438,0.05562
2,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0,0,0.91906,0.94645,0.05355
3,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0,0,0.97978,0.98753,0.01247
4,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0,0,0.90595,0.93229,0.06771
5,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0,0,0.89526,0.91625,0.08375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101499,NaN,0.282653,24,0,0.068522,1400.0,5,0,0,0,0.0,0,0.96958,0.98109,0.01891
101500,NaN,0.922156,36,3,0.934217,7615.0,8,0,2,0,4.0,0,0.66054,0.70392,0.29608
101501,NaN,0.081596,70,0,836.000000,NaN,3,0,0,0,NaN,0,0.99448,0.99612,0.00387


In [6]:
df_to_send = df[['1 probability']]
df_to_send.index.names=['Id']
df_to_send.rename(columns={"1 probability":"Probability"}, inplace=True)
df_to_send

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,Probability
Id,
1,0.05562
2,0.05355
3,0.01247
4,0.06771
5,0.08375
...,...
101499,0.01891
101500,0.29608
101501,0.00387


In [7]:
df_to_send.to_csv("./kaggle_prediction.csv")

In [8]:
help(kaggle.api.competition_submit)
kaggle.api.authenticate()
kaggle.api.competition_submit("./kaggle_prediction.csv", "my message to you Rudy", "GiveMeSomeCredit")

Help on method competition_submit in module kaggle.api.kaggle_api_extended:

competition_submit(file_name, message, competition, quiet=False) method of kaggle.api.kaggle_api_extended.KaggleApi instance
    submit a competition!
    
    Parameters
    file_name: the competition metadata file
    message: the submission description
    competition: the competition name
    quiet: suppress verbose output (default is False)



100%|██████████| 1.65M/1.65M [00:05<00:00, 293kB/s] 


Successfully submitted to Give Me Some Credit